In [ ]:
from google import genai
from pydantic import BaseModel, Field
from typing import Union, Literal
from datetime import datetime

API_KEY=""

In [3]:
class GoalDefinition(BaseModel):
    goal: str = Field(description="The specific goal to achieve (e.g., 'Run a marathon').")
    metric: str = Field(description="The **objective, quantifiable** metric that determines goal completion (e.g., 'Finish in under 4 hours', 'Save $5,000').")
    purpose: str = Field(description="The underlying motivation behind the goal.")
    deadline: datetime = Field(description="The exact date and time the goal must be completed by, formatted as a standard datetime object.")
    status: Literal['details_extracted'] = Field(
        'details_extracted', 
        description="A fixed string indicating successful, quantifiable extraction."
    )

class FollowUpDefinition(BaseModel):
    """A response model when a key piece of information is missing OR subjective."""
    status: Literal['follow_up_required'] = Field(
        'follow_up_required', 
        description="A fixed string indicating missing or subjective data."
    )
    issue_type: Literal['missing_field', 'subjective_metric'] = Field(
        description="Indicates whether the issue is a missing required field or a metric that needs quantification."
    )
    missing_or_subjective_fields: list[str] = Field(
        description="A list of fields that need clarification (e.g., ['metric', 'deadline'])."
    )
    question_to_user: str = Field(
        description="A specific, focused question to obtain the missing or quantifiable information."
    )

ResponseSchema = Union[GoalDefinition, FollowUpDefinition]

In [6]:
SYSTEM_INSTRUCTION = (
    "You are an expert goal-setting assistant. Your task is to structure the user's input into a defined format."
    "You may ask follow up questions that will help in planning for the goal."
    "For a successful output ('GoalDefinition'), you MUST extract: 'goal', 'metric', 'purpose', and a precise 'deadline' (format as YYYY-MM-DDTHH:MM:SS).\n"
    "**CRITICAL RULE FOR METRIC:** The 'metric' MUST be objective and quantifiable (e.g., 'score 85%', '20 pushups', 'finish $500 product'). "
    "Subjective metrics (e.g., 'performance ready', 'feel better', 'fluent') are NOT acceptable.\n\n"
    "**Conditional Output Rules:**\n"
    "1. **SUCCESS:** If all four required fields are present AND the 'metric' is quantifiable, use the 'GoalDetails' schema.\n"
    "2. **FOLLOW-UP (Missing):** If any required field is missing, use the 'FollowUpDefinition' schema, set 'issue_type' to 'missing_field', and ask for the missing data.\n"
    "3. **FOLLOW-UP (Subjective):** If the 'metric' is present but subjective, use the 'FollowUpDefinition' schema, set 'issue_type' to 'subjective_metric', and ask the user to make the metric objective and measurable."
)

In [7]:
client = genai.Client(api_key=API_KEY)

# 1. Create a Chat object with the necessary structured config
chat = client.chats.create(
    model='gemini-2.5-flash',
    config={
        "system_instruction": SYSTEM_INSTRUCTION,
        "response_mime_type": "application/json",
        "response_schema": ResponseSchema,
    },
)

user_input = input("I want to: ")
response = chat.send_message(user_input)
results = response.parsed

while isinstance(results, FollowUpDefinition):
    print(f"Issue Type: {results.issue_type}")
    new_input = input(f"{results.question_to_user} ")
    
    response = chat.send_message(new_input)
    results = response.parsed

print("---------------------------------")
print(f"Goal: {results.goal}")
print(f"Metric: {results.metric}")      
print(f"Purpose: {results.purpose}")      
print(f"Deadline: {results.deadline.strftime('%Y-%m-%d %H:%M')}")
print(f"Status: {results.status}")

Issue Type: missing_field
Issue Type: missing_field
Issue Type: missing_field
---------------------------------
Goal: learn coding
Metric: complete 3 full-stack projects in a public repository
Purpose: get a job
Deadline: 2024-12-31 23:59
Status: details_extracted
